# 数据聚合与分组运算

## GroupBy机制

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
....: 'key2' : ['one', 'two', 'one', 'two', 'one'],
....: 'data1' : np.random.randn(5),
....: 'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.031033,1.572287
1,a,two,-1.012501,0.270636
2,b,one,-1.119238,-0.568033
3,b,two,-0.349595,0.216172
4,a,one,-1.692594,-0.017403


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.912043
b   -0.734416
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.861814
      two    -1.012501
b     one    -1.119238
      two    -0.349595
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.861814,-1.012501
b,-1.119238,-0.349595


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states,years]).mean()

California  2005   -1.012501
            2006   -1.119238
Ohio        2005   -0.190314
            2006   -1.692594
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.912043,0.608507
b,-0.734416,-0.175930


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.861814  0.777442
     two  -1.012501  0.270636
b    one  -1.119238 -0.568033
     two  -0.349595  0.216172

In [11]:
df.groupby(['key1', 'key2']).size()  #返回个分组的大小

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [14]:
#对分组进行迭代
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.031033  1.572287
1    a  two -1.012501  0.270636
4    a  one -1.692594 -0.017403
b
  key1 key2     data1     data2
2    b  one -1.119238 -0.568033
3    b  two -0.349595  0.216172


In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.031033  1.572287
4    a  one -1.692594 -0.017403
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.012501  0.270636
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.119238 -0.568033
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.349595  0.216172


In [16]:
pieces = dict(list(df.groupby('key1')))
pieces['b']    #将数据片段做成字典

,key1,key2,data1,data2
2,b,one,-1.119238,-0.568033
3,b,two,-0.349595,0.216172


In [17]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped=df.groupby(df.dtypes,axis=1)  #在轴1进行分组
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.031033  1.572287
1 -1.012501  0.270636
2 -1.119238 -0.568033
3 -0.349595  0.216172
4 -1.692594 -0.017403
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [19]:
#选取一列或列的子集
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.777442
     two   0.270636
b    one  -0.568033
     two   0.216172

In [20]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one     0.777442
      two     0.270636
b     one    -0.568033
      two     0.216172
Name: data2, dtype: float64

In [23]:
#通过字典或Sreies进行分组
people = pd.DataFrame(np.random.randn(5, 5),
columns=['a', 'b', 'c', 'd', 'e'],
index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [24]:
people

,a,b,c,d,e
Joe,-0.175546,-0.167073,2.607667,-2.111362,0.855794
Steve,-0.306153,-0.065160,-0.678019,-1.291726,1.646544
Wes,-1.197586,-0.405532,-1.929203,-0.627910,-0.059760
Jim,-0.848298,-1.021280,-0.053912,-0.997671,-0.518635
Travis,-0.788273,-0.196900,0.619644,1.316053,2.512760


In [26]:
people.iloc[2:3,[1,2]]=np.nan

In [27]:
people

,a,b,c,d,e
Joe,-0.175546,-0.167073,2.607667,-2.111362,0.855794
Steve,-0.306153,-0.065160,-0.678019,-1.291726,1.646544
Wes,-1.197586,NaN,NaN,-0.627910,-0.059760
Jim,-0.848298,-1.021280,-0.053912,-0.997671,-0.518635
Travis,-0.788273,-0.196900,0.619644,1.316053,2.512760


In [28]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
 'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [29]:
by_column = people.groupby(mapping, axis=1)

In [30]:
by_column.sum()

,blue,red
Joe,0.496304,0.513175
Steve,-1.969745,1.275231
Wes,-0.627910,-1.257345
Jim,-1.051583,-2.388213
Travis,1.935697,1.527587


In [31]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [33]:
#通过函数进行分组
people.groupby(len).sum()

,a,b,c,d,e
3,-2.221429,-1.188353,2.553755,-3.736944,0.277399
5,-0.306153,-0.065160,-0.678019,-1.291726,1.646544
6,-0.788273,-0.196900,0.619644,1.316053,2.512760


In [34]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.197586 -0.167073  2.607667 -2.111362 -0.059760
  two -0.848298 -1.021280 -0.053912 -0.997671 -0.518635
5 one -0.306153 -0.065160 -0.678019 -1.291726  1.646544
6 two -0.788273 -0.196900  0.619644  1.316053  2.512760

In [35]:
#根据索引级别分组
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
[1, 3, 5, 1, 3]],names=['cty', 'tenor'])

In [36]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [37]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.639276  0.060674 -0.437680 -0.175744  2.323131
1      0.584930 -0.956976 -1.014470 -0.147909  0.455950
2      1.366348  1.118003  0.889796 -0.370541 -1.138179
3      0.415361 -1.645088  0.614501 -0.022458 -2.867054

In [39]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

In [40]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a   -0.227327
b   -0.426559
Name: data1, dtype: float64

In [41]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.661561,1.589690
b,0.769643,0.784205


In [42]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.912043  0.835324 -1.692594 -1.352548 -1.012501 -0.521767   
b      2.0 -0.734416  0.544220 -1.119238 -0.926827 -0.734416 -0.542005   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.031033   3.0  0.608507  0.846993 -0.017403  0.126616  0.270636   
b    -0.349595   2.0 -0.175930  0.554517 -0.568033 -0.371982 -0.175930   

                          
           75%       max  
key1                      
a     0.921461  1.572287  
b     0.020121  0.216172

In [2]:
#面向列的多函数应用   agg[]中写函数，（）用来命名
 tips = pd.read_csv('examples/tips.csv')

In [5]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [45]:
grouped = tips.groupby(['day', 'smoker'])

In [46]:
grouped_pct=grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [47]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [48]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])  #命名

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [49]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)   

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [50]:
#对⼀个列或不同的列应⽤不同的函数，向agg传⼊⼀个从列名映射到函数的字典
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [51]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [52]:
#以“没有航索引”的形式返回聚合函数
 tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply:一般性的“拆分-应用-合并”

In [3]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [6]:
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [7]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [8]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [9]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [10]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [11]:
#禁止分组键
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [13]:
#分位数和桶分析
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})

In [14]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (0.266, 1.755]
1     (0.266, 1.755]
2    (-1.222, 0.266]
3    (-1.222, 0.266]
4     (0.266, 1.755]
5     (1.755, 3.243]
6     (0.266, 1.755]
7    (-1.222, 0.266]
8    (-1.222, 0.266]
9     (0.266, 1.755]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.716, -1.222] < (-1.222, 0.266] < (0.266, 1.755] < (1.755, 3.243]]

In [16]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [17]:
grouped=frame.data2.groupby(quartiles)

In [18]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.716, -1.222]",97.0,1.844904,-0.159800,-2.583709
"(-1.222, 0.266]",496.0,2.790685,0.035492,-2.576059
"(0.266, 1.755]",362.0,3.156524,0.044220,-2.790373
"(1.755, 3.243]",45.0,2.545073,0.203708,-2.653744


In [19]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.844904,-0.141177,-2.583709
1,100.0,2.376450,-0.032660,-2.576059
2,100.0,2.648790,0.159867,-2.179584
3,100.0,2.436455,0.021443,-1.665912
4,100.0,2.790685,-0.007905,-2.003229
5,100.0,2.469645,-0.009160,-2.113731
6,100.0,2.341220,0.112375,-2.189776
7,100.0,3.156524,0.087723,-2.777939
8,100.0,1.983775,0.022099,-2.253185


In [20]:
#用特定于分组的值填充缺失值
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.452136
2         NaN
3   -0.038929
4         NaN
5    0.219445
dtype: float64

In [21]:
s.fillna(s.mean())

0   -0.090540
1   -0.452136
2   -0.090540
3   -0.038929
4   -0.090540
5    0.219445
dtype: float64

In [22]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.315450
New York     -1.632116
Vermont       0.004860
Florida      -1.476393
Oregon        0.777192
Nevada       -0.074150
California    0.881154
Idaho        -1.506030
dtype: float64

In [23]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [24]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.315450
New York     -1.632116
Vermont            NaN
Florida      -1.476393
Oregon        0.777192
Nevada             NaN
California    0.881154
Idaho              NaN
dtype: float64

In [25]:
data.groupby(group_key).mean()

East   -0.931020
West    0.829173
dtype: float64

In [27]:
fill_mean=lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.315450
New York     -1.632116
Vermont      -0.931020
Florida      -1.476393
Oregon        0.777192
Nevada        0.829173
California    0.881154
Idaho         0.829173
dtype: float64

In [28]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.315450
New York     -1.632116
Vermont       0.500000
Florida      -1.476393
Oregon        0.777192
Nevada       -1.000000
California    0.881154
Idaho        -1.000000
dtype: float64

In [34]:
#随机采样和排列
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [35]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [37]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

4H      4
5D      5
10S    10
JH     10
7S      7
dtype: int64

In [38]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  4C    4
   7C    7
D  4D    4
   2D    2
H  4H    4
   7H    7
S  6S    6
   2S    2
dtype: int64

In [40]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=1)

9C    9
6D    6
6H    6
4S    4
dtype: int64

In [41]:
#分组加权平均数和相关系数
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
 'b', 'b', 'b', 'b'], 'data': np.random.randn(8),
'weights': np.random.rand(8)})

In [42]:
df

,category,data,weights
0,a,2.079409,0.365330
1,a,0.166849,0.003132
2,a,-0.402971,0.537545
3,a,1.609684,0.357221
4,b,0.732110,0.636788
5,b,-0.065408,0.110492
6,b,-1.781447,0.140221
7,b,0.362686,0.720657


In [43]:
grouped=df.groupby('category')
get_wavg=lambda g: np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.885502
b    0.292600
dtype: float64

In [44]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
.....: index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [45]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [46]:
spx_corr=lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()

In [47]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [48]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [51]:
#组级别的线性回归
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [52]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 透视表和交叉表

In [53]:
tips.pivot_table(index=['day', 'smoker']) #默认聚合类型为平均数

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [56]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [57]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
.....: columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [59]:
#使⽤其他的聚合函数，将其传给aggfunc
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [60]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [62]:
#交叉表
 pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244